Module imports

In [18]:
# Import modules

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch
import pandas as pd

Data import

In [72]:
# Read data into DataFrames

# Emissions data
nrg_emi_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="CO2 Emissions from Energy", header=2, index_col=0)
# The sheet called "Natural Gas Flaring" is already a part of the calculations for the sheet called "CO2 from Flaring"
flar_emi_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="CO2 from Flaring", header=2, index_col=0)
equi_emi_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="CO2e Methane, Process emissions", header=2, index_col=0)

# Renewable energy data
hydro_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="Hydro Consumption - EJ", header=2, index_col=0)
solar_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="Solar Consumption - EJ", header=2, index_col=0)
wind_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="Wind Consumption - EJ", header=2, index_col=0)
geo_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="Geo Biomass Other - EJ", header=2, index_col=0)
biofuel_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="Biofuels consumption - PJ", header=2, index_col=0, nrows=47)

Programmatic data processing

In [77]:
def processData(df:pd.DataFrame):
    """
    Get an excel sheet ready for conversion to numpy arrays.

    Parameters:
    - df (pd.DataFrame): a dataframe containing an excel sheet
    """
    #------------------------------ 
    # Remove all irrelevant columns
    #------------------------------

    # Remove all data from before 1990
    # Find the index of the "1990" column
    drop_indx = list(df.columns).index(1990)
    # Get the column labels of all columns left of "1990"
    drop_cols = [df.columns[num] for num in np.arange(0, drop_indx)]
    df = df.drop(columns=drop_cols)

    # Remove data on growth-rate and share
    # Get the column labels of the target columns
    drop_cols = [df.columns[num] for num in [-3, -2, -1]]
    df = df.drop(columns=drop_cols)

    #---------------------------
    # Remove all irrelevant rows
    #---------------------------

    # Remove all rows with any empty cells
    # 0 doesn't make an empty cell
    df = df.dropna()

    # Remove all "Total" and "Other" rows
    # In addition, OECD, Non-OECD, the EU, and the USSR
    # Rationale for removing "Other" rows - some countries in some excel sheets appear
    # individually, but are lumped into an "Other" row in other sheets.
    # There's no possible way for me to know which portions of an
    # "Other" row value belongs to which countries.
    drop_rows = []
    keywords = ["Total", "Other", "OECD", "European Union", "USSR"]
    for row in df.index:
        # Mark a row for dropping if it contains any of the keywords
        if any(keyword in row for keyword in keywords):
            drop_rows.append(row)
    df = df.drop(index=drop_rows)

    # --------------------
    # Convert any PJ to EJ
    # --------------------

    # 1 PJ = 0.001 EJ
    # 1 EJ = 1000 PJ
    if df.index.name == "Petajoules":
        df = df * 0.001

    return df

# tonnes = metric ton = 1000 kg


In [78]:
def rowIndices(df:pd.DataFrame):
    """
    Return the row labels of a pd.DataFrame

    Parameters:
    - df (pd.DataFrame): a dataframe containing an excel sheet
    """

    # for row in df.index:
    #     print(row)

    return [row for row in df.index]


In [79]:
# Process dataframes

nrg_emi_df = processData(nrg_emi_df)
flar_emi_df = processData(flar_emi_df)
equi_emi_df = processData(equi_emi_df)

hydro_df = processData(hydro_df)
solar_df = processData(solar_df)
wind_df = processData(wind_df)
geo_df = processData(geo_df)
biofuel_df = processData(biofuel_df)

In [80]:
# Convert to numpy arrays

nrg_emi = nrg_emi_df.to_numpy()
flar_emi = flar_emi_df.to_numpy()
equi_emi = equi_emi_df.to_numpy()
hydro = hydro_df.to_numpy()
solar = solar_df.to_numpy()
wind = wind_df.to_numpy()
geo = geo_df.to_numpy()
biofuel = biofuel_df.to_numpy()

# Get row indices of dataframes

nrg_emi_indices = rowIndices(nrg_emi_df)
flar_emi_indices = rowIndices(flar_emi_df)
equi_emi_indices = rowIndices(equi_emi_df)
hydro_indices = rowIndices(hydro_df)
solar_indices = rowIndices(solar_df)
wind_indices = rowIndices(wind_df)
geo_indices = rowIndices(geo_df)
biofuel_indices = rowIndices(biofuel_df)

In [81]:
# Separate data into combinations of countries/regions and years

from collections import Counter

print(len(nrg_emi_indices))
print(len(flar_emi_indices))
print(len(equi_emi_indices))

print(len(hydro_indices))
print(len(solar_indices))
print(len(wind_indices))
print(len(geo_indices))
print(len(biofuel_indices))

print(nrg_emi_indices)
print(flar_emi_indices)
print(biofuel_indices)

83
49
83
83
83
83
83
24
['Canada', 'Mexico', 'US', 'Argentina', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Peru', 'Trinidad & Tobago', 'Venezuela', 'Central America', 'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine', 'United Kingdom', 'Azerbaijan', 'Belarus', 'Kazakhstan', 'Russian Federation', 'Turkmenistan', 'Uzbekistan', 'Iran', 'Iraq', 'Israel', 'Kuwait', 'Oman', 'Qatar', 'Saudi Arabia', 'United Arab Emirates', 'Algeria', 'Egypt', 'Morocco', 'South Africa', 'Eastern Africa', 'Middle Africa', 'Western Africa', 'Australia', 'Bangladesh', 'China', 'China Hong Kong SAR', 'India', 'Indonesia', 'Japan', 'Malaysia', 'New Zealand', 'Pakistan', 'Philippines', 'Singapore', 'S